In [24]:
from pyspark.sql import SparkSession
import csv
import pandas as pd

spark = SparkSession.builder.master("local").appName("lopputyo").getOrCreate()

df = spark.read.csv("merged19_20_21.csv", header=True, inferSchema=True)
df.show(vertical=True)

df.registerTempTable("example")


-RECORD 0----------------------------------
 id                 | 0                    
 aikatyyppi         | Vuosikertymä         
 kuukausi_nro       | 12                   
 vuosikuukausi      | 201912               
 vuosi              | 2019                 
 kunta_nro          | 20                   
 kunta_nimi         | Akaa                 
 ikaryhma           | 25-29                
 sukupuoli          | Mies                 
 etuus              | Työmarkkinatukiet... 
 korvausperuste     | Työllistymistä ed... 
 saaja_lkm          | 19                   
 korvattu_paiva_lkm | 3158                 
 maksettu_eur       | 120106.43            
-RECORD 1----------------------------------
 id                 | 1                    
 aikatyyppi         | Vuosikertymä         
 kuukausi_nro       | 12                   
 vuosikuukausi      | 201912               
 vuosi              | 2019                 
 kunta_nro          | 19                   
 kunta_nimi         | Aura      

c:\Lipasto\MassData23\mass_data_analysis_2023\venv\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [25]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- aikatyyppi: string (nullable = true)
 |-- kuukausi_nro: integer (nullable = true)
 |-- vuosikuukausi: integer (nullable = true)
 |-- vuosi: integer (nullable = true)
 |-- kunta_nro: integer (nullable = true)
 |-- kunta_nimi: string (nullable = true)
 |-- ikaryhma: string (nullable = true)
 |-- sukupuoli: string (nullable = true)
 |-- etuus: string (nullable = true)
 |-- korvausperuste: string (nullable = true)
 |-- saaja_lkm: integer (nullable = true)
 |-- korvattu_paiva_lkm: integer (nullable = true)
 |-- maksettu_eur: double (nullable = true)



In [26]:
df.groupBy('etuus').count().orderBy('count',ascending=False).show()

+--------------------+-------+
|               etuus|  count|
+--------------------+-------+
|            Yhteensä|1464095|
|Työmarkkinatukiet...|1373934|
|     Työmarkkinatuki|1373833|
|      Peruspäiväraha| 570001|
|      Kotoutumistuki|   2476|
+--------------------+-------+



In [49]:
new_df = df.withColumn("maksu_per_hlo", (df["maksettu_eur"]/df["saaja_lkm"])).orderBy('id',ascending=False).filter(df["aikatyyppi"] == "Kuukausi")
new_df.show()


+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+---------------+--------------------+---------+------------------+------------+------------------+
|     id|aikatyyppi|kuukausi_nro|vuosikuukausi|vuosi|kunta_nro| kunta_nimi|ikaryhma|sukupuoli|          etuus|      korvausperuste|saaja_lkm|korvattu_paiva_lkm|maksettu_eur|     maksu_per_hlo|
+-------+----------+------------+-------------+-----+---------+-----------+--------+---------+---------------+--------------------+---------+------------------+------------+------------------+
|4784338|  Kuukausi|          12|       202112| 2021|       78|      Hanko|   45-49|     Mies|Työmarkkinatuki|-Kuntouttava työt...|        7|               133|     5284.24| 754.8914285714285|
|4784337|  Kuukausi|          12|       202112| 2021|       91|   Helsinki|   45-49|     Mies|Työmarkkinatuki|-Kuntouttava työt...|       55|              1149|    46989.36|           854.352|
|4784336|  Kuukausi|          12|  

In [50]:
new_df.select("saaja_lkm", "maksettu_eur", "maksu_per_hlo").describe().show()

+-------+-----------------+------------------+------------------+
|summary|        saaja_lkm|      maksettu_eur|     maksu_per_hlo|
+-------+-----------------+------------------+------------------+
|  count|          2007266|           2007266|           1958648|
|   mean|27.01970640662473|20637.222865272808| 732.9145452507237|
| stddev| 92.9363397265622| 72886.42450175941|1134.5196466522586|
|    min|                0|        -305996.29|        -215174.01|
|    max|             3760|         2785132.2|          16037.11|
+-------+-----------------+------------------+------------------+



In [51]:
new_df.select("kuukausi_nro", "maksu_per_hlo").groupBy('kuukausi_nro').avg().orderBy("kuukausi_nro").show()

+------------+-----------------+------------------+
|kuukausi_nro|avg(kuukausi_nro)|avg(maksu_per_hlo)|
+------------+-----------------+------------------+
|           1|              1.0| 726.9060755497777|
|           2|              2.0|  703.142235902703|
|           3|              3.0| 747.2579507203649|
|           4|              4.0| 748.8572327196003|
|           5|              5.0| 743.7531116470808|
|           6|              6.0| 715.1470000733383|
|           7|              7.0|  745.350990892491|
|           8|              8.0| 697.1801402705964|
|           9|              9.0| 720.1780101994516|
|          10|             10.0| 735.4666687260947|
|          11|             11.0| 724.3641439753526|
|          12|             12.0| 784.7651191810176|
+------------+-----------------+------------------+



In [52]:
new_df.select("ikaryhma", "maksu_per_hlo").groupBy('ikaryhma').avg().orderBy("ikaryhma").show()

+-------------+------------------+
|     ikaryhma|avg(maksu_per_hlo)|
+-------------+------------------+
|        17-19| 667.8246949503166|
|        20-24| 714.8403877688473|
|        25-29| 761.7546401057336|
|        30-34| 794.9645386138585|
|        35-39| 817.3439506175276|
|        40-44| 812.6604219503253|
|        45-49| 787.8406128457792|
|        50-54|  752.552064522686|
|        55-59| 727.1541813808893|
|        60-64| 714.9705868728151|
|        65-67| 535.9984415595703|
|Tieto puuttuu| 619.9086500742875|
+-------------+------------------+



In [53]:
new_df.select("vuosi", "maksu_per_hlo").groupBy('vuosi').avg().orderBy("vuosi").show()

+-----+----------+------------------+
|vuosi|avg(vuosi)|avg(maksu_per_hlo)|
+-----+----------+------------------+
| 2019|    2019.0| 717.0590836157269|
| 2020|    2020.0| 738.0551154443735|
| 2021|    2021.0| 743.8381485453482|
+-----+----------+------------------+

